In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from random import randint
import random
from proxy_checker import ProxyChecker

In [ ]:
# If you are planning to go through user input

user_input = []

while True:
    user_dict = {}
    search = input('Please input your wine name,vintage and quantity, all separated by a comma.\n')
    if len(search.split(',')) != 3:
        print('Apologies, your input wasn\'t in the correct format! \n')
        continue
    else:
        result = search.split(',')
    user_dict['name'] = result[0].replace(' ', '_')
    user_dict['vintage'] = result[1].replace(' ','')
    user_dict['qty'] = result[2].replace(' ','')
    user_input.append(user_dict)
    confirmation = input('If that is all, please input confirm, else input continue.\n')
    if confirmation == 'confirm':
        break
    else:
        continue
        


In [ ]:
# If you are storing in a wine excel sheet previously

user_input_df = pd.read_excel('wine.xlsx', usecols = 'A:C')

user_input = [{'name':x.replace(' ','_'), 'vintage':str(y), 'qty':z}
               for x,y,z in zip(user_input_df['Wine'],user_input_df['Vintage'],user_input_df['Quantity'])]

In [30]:
user_input

[{'name': 'petrus', 'vintage': '2000', 'qty': '1'}]

In [31]:
overall_df = pd.read_csv('wine_price_list.csv')

agents = user_agents = [
    'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15']

proxies = [{'https': 'http://159.203.35.22:8080'},
           {'https': 'http://5.161.181.70:8080'},
          ]


for i in range(len(user_input)):
        
    ## Parameters for search
    base_URL = 'http://www.wine-searcher.com/find/'
    name = user_input[i]['name']
    if randint(0,100) >= 75:
        name = name.replace('_', '+')
    vintage = user_input[i]['vintage']
    noAuction = '-/ni'
    country = 'singapore'
    qty = int(user_input[i]['qty'])
    
    agent = {"User-Agent": random.choice(user_agents)}
    proxy = random.choice(proxies)

    URL = base_URL + name + '/' + vintage + '/' + country + '/' + noAuction
    
    time.sleep(randint(30,60))
    
    
    ## GET request
    page = requests.get(URL, headers = agent, proxies = proxy, verify = False)

    
    ## Analyzing and decoding HTML
    soup = BeautifulSoup(page.content, "html.parser")

    results = soup.body.main.find(class_ = 'container pt-3 pb-3 px-md-0 pt-0')\
    .find(class_ = 'js-offers-container').find_all(class_ = 'offer-card__container')
    
    curated_results = []
    chosen_result = ''
    
    ## Choosing bottles > magnum/case, but if you are a collector you can convert this to case very easily
    
    for i in results:
        merchant = i.find(class_ = 'col1').find(class_ = 'offer-card__merchant-name').text 
        bottle = i.find(class_ = 'badge-bottle border badge-lg badge badge-pill')
        magnum = i.find(class_ = 'badge-other border badge-lg badge badge-pill')
        if bottle:
            chosen_result = i
            break
        elif merchant == 'FINE+RARE Singapore' or magnum:
            pass
        else:
            curated_results.append(i)
    
    if chosen_result == '':
        if len(curated_results) == 0:
            continue
        chosen_result = curated_results[0] 
    
    
    ## Getting required price and calculations after
    
    price = chosen_result.find(class_ = 'col2 pl-0 pl-lg-3')\
    .find(class_ = 'price__detail price__detail_main small').find(class_ = 'price__integer-part align-middle').text
    
    if chosen_result.find(class_ = 'badge-cases border badge-lg badge badge-pill'):
        bottles = chosen_result.find(class_ = 'badge-cases border badge-lg badge badge-pill').text
    elif chosen_result.find(class_ = 'badge-bottle border badge-lg badge badge-pill'):
        bottles = chosen_result.find(class_ = 'badge-bottle border badge-lg badge badge-pill').text
    print(name,price)
    print(bottles)
    
    ## Checking for duty/sales tax
    
    condition = chosen_result.find(class_ = 'col2 pl-0 pl-lg-3').find(class_ = 'price__tax-status text-muted small').text

    ## Converting price to actual price based on type of case
    
    if price.find(',') != -1:
        price = price.replace(',', '')
    if bottles == 'Case of 6 Btls':
        actual_price = round(int(price) / 6, 0)
    elif bottles == 'Case of 12 Btls':
        actual_price = round(int(price) / 12, 0)
    elif bottles == 'Bottle (750ml)':
        actual_price = round(int(price), 0)
        
    ## Assuming it's 15% alcohol

    if condition == 'ex. duty and sales tax':
        actual_price = round(1.08* (actual_price + 8.25),2)
    elif condition == 'inc. 8% sales tax':
        actual_price = round(actual_price,2)
        
    names = ['Name', 'Vintage', 'Qty', 'Price']
    index = [name,vintage,qty,actual_price]

    ind = {}

    j = 0 
    
    ## Updating the dataframe if there is already such bottles in inventory
    for i in names:
        ind[i] = index[j]
        j = j + 1

    if overall_df[(overall_df['Vintage'] == int(vintage)) & (overall_df['Name'] == name)].empty:
        overall_df.loc[len(overall_df)] = ind
    else:
        overall_df.loc[(overall_df['Vintage'] == int(vintage)) & (overall_df['Name'] == name), 'Qty'] += ind['Qty']
        overall_df.loc[(overall_df['Vintage'] == int(vintage)) & (overall_df['Name'] == name), 'Price'] = ind['Price']

C:\Users\User\anaconda3\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host '159.203.35.22'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


petrus 8,919
Bottle (750ml)


In [32]:
overall_df

,Name,Vintage,Qty,Price
0,petrus,2000,2,9641.43


In [29]:
overall_df.to_csv('wine_price_list.csv',index = False)